In [1]:
# ! rm -rf PyTorch-Architectures/
! git clone https://github.com/vishal-burman/PyTorch-Architectures.git

Cloning into 'PyTorch-Architectures'...
remote: Enumerating objects: 202, done.
remote: Counting objects: 100% (202/202), done.
remote: Compressing objects: 100% (125/125), done.
remote: Total 633 (delta 114), reused 144 (delta 69), pack-reused 431
Receiving objects: 100% (633/633), 8.38 MiB | 5.20 MiB/s, done.
Resolving deltas: 100% (390/390), done.


In [2]:
%cd PyTorch-Architectures/modeling_xlm/

/kaggle/working/PyTorch-Architectures/modeling_xlm


In [4]:
! pip install datasets
! pip install transformers

     |████████████████████████████████| 147 kB 193 kB/s eta 0:00:01
     |████████████████████████████████| 243 kB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 17.7 MB 2.2 MB/s eta 0:00:01     |███████████████████████         | 12.7 MB 2.2 MB/s eta 0:00:03
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 0.16.0
    Uninstalling pyarrow-0.16.0:
      Successfully uninstalled pyarrow-0.16.0
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [5]:
from os import path
import time
import csv
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import XLMTokenizer
from config_xlm import XLMConfig
from model import XLMWithLMHeadModel
from datasets import load_dataset
from transformers import top_k_top_p_filtering

dataset = load_dataset('cnn_dailymail', '3.0.0')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/0128610a44e10f25b4af6689441c72af86205282d26399642f7db38fa7535602. Subsequent calls will reuse this data.


In [6]:
# Defining the model and tokenizer
config = XLMConfig()
config.n_layers = 6
tokenizer = XLMTokenizer.from_pretrained('xlm-mlm-en-2048')
model = XLMWithLMHeadModel(config).to(device)

In [7]:
total_params = sum(p.numel() for p in model.parameters())
print("Total Parameters = ", total_params)

Total Parameters =  50076097


In [8]:
class CustomDataset(Dataset):
    def __init__(self, texts, tokenizer, seq_length=64):
        self.tokenizer = tokenizer
        self.texts = texts
        self.seq_length = seq_length
        self.train_list = []
        self.build()

    def __len__(self):
        return len(self.train_list)

    def __getitem__(self, index):
        ids = self.train_list[index]['input_ids']
        mask = self.train_list[index]['attention_mask']

        return{
                'ids': torch.tensor(ids, dtype=torch.long),
                'mask': torch.tensor(ids, dtype=torch.long),
                }

    def build(self):
        for t in self.texts:
            self.train_list.append(tokenizer(t, max_length=self.seq_length, pad_to_max_length=True, truncation=True))

In [9]:
! rm -rf *.pt

In [10]:
if path.exists('train_dataset.pt') and path.exists('valid_dataset.pt'):
  print("Datasets is already present!")
  train_dataset = torch.load('train_dataset.pt')
  valid_dataset = torch.load('valid_dataset.pt')

else:
  texts_train = dataset['train']['article'][:10000]

  texts_valid = dataset['validation']['article'][:1000]

  start_time = time.time()
  train_dataset = CustomDataset(texts_train, tokenizer, seq_length=256)
  valid_dataset = CustomDataset(texts_valid, tokenizer)
  torch.save(train_dataset, 'train_dataset.pt')
  torch.save(valid_dataset, 'valid_dataset.pt')
  print("Dataset Conversion Done!!")
  print("Time Taken = ", (time.time() - start_time)/60)

Dataset Conversion Done!!
Time Taken =  4.767487871646881


In [11]:
BATCH_SIZE = 16
LR = 2e-04
EPOCHS = 10

train_loader = DataLoader(dataset=train_dataset, shuffle=True, batch_size=BATCH_SIZE)
valid_loader = DataLoader(dataset=valid_dataset, shuffle=False, batch_size=BATCH_SIZE)
print("Length of Train DataLoader: ", len(train_loader))
print("Length of Valid DataLoader: ", len(valid_loader))

optimizer = torch.optim.Adam(model.parameters(), lr=LR)

Length of Train DataLoader:  625
Length of Valid DataLoader:  63


In [12]:
def compute_loss(model, data_loader, device):
  total_loss = 0
  model.eval()
  with torch.set_grad_enabled(False):
    for sample in data_loader:
      ids = sample['ids'].to(device)
      mask = sample['mask'].to(device)
      labels = sample['ids'].to(device)

      outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
      loss = outputs[0]
      total_loss += loss.item()
  return (total_loss / len(data_loader))

start_time = time.time()
for epoch in range(EPOCHS):
  model.train()
  for idx, sample in enumerate(train_loader):
    ids = sample['ids'].to(device)
    mask = sample['mask'].to(device)
    labels = sample['ids'].to(device)
    
    optimizer.zero_grad()
    
    logits = model(input_ids=ids, attention_mask=mask, labels=labels)
    loss = logits[0]

    # LOGGING
    if idx % 100 == 0:
      print("Batch: %04d/%04d || Epoch: %03d/%03d" % (idx, len(train_loader), epoch+1, EPOCHS))

    loss.backward()
    optimizer.step()

  model.eval()
  with torch.set_grad_enabled(False):
    train_loss = compute_loss(model, train_loader, device)
    valid_loss = compute_loss(model, valid_loader, device)
    print("Train Loss: %.3f" % (train_loss))
    print("Valid Loss: %.3f" % (valid_loss))
  elapsed_epoch_time = (time.time() - start_time) / 60
  print("Epoch Elapsed Time: %d mins" % (elapsed_epoch_time))
total_training_time = (time.time() - start_time) / 60
print("Total Training Time: %d mins" % (elapsed_epoch_time))

/opt/conda/lib/python3.7/site-packages/torch/nn/_reduction.py:14: UserWarning: reduction='elementwise_mean' is deprecated, please use reduction='mean' instead.
  warnings.warn("reduction='elementwise_mean' is deprecated, please use reduction='mean' instead.")


Batch: 0000/0625 || Epoch: 001/010
Batch: 0100/0625 || Epoch: 001/010
Batch: 0200/0625 || Epoch: 001/010
Batch: 0300/0625 || Epoch: 001/010
Batch: 0400/0625 || Epoch: 001/010
Batch: 0500/0625 || Epoch: 001/010
Batch: 0600/0625 || Epoch: 001/010
Train Loss: 209.302
Valid Loss: 247.115
Epoch Elapsed Time: 2 mins
Batch: 0000/0625 || Epoch: 002/010
Batch: 0100/0625 || Epoch: 002/010
Batch: 0200/0625 || Epoch: 002/010
Batch: 0300/0625 || Epoch: 002/010
Batch: 0400/0625 || Epoch: 002/010
Batch: 0500/0625 || Epoch: 002/010
Batch: 0600/0625 || Epoch: 002/010
Train Loss: 64.651
Valid Loss: 96.894
Epoch Elapsed Time: 5 mins
Batch: 0000/0625 || Epoch: 003/010
Batch: 0100/0625 || Epoch: 003/010
Batch: 0200/0625 || Epoch: 003/010
Batch: 0300/0625 || Epoch: 003/010
Batch: 0400/0625 || Epoch: 003/010
Batch: 0500/0625 || Epoch: 003/010
Batch: 0600/0625 || Epoch: 003/010
Train Loss: 37.802
Valid Loss: 64.964
Epoch Elapsed Time: 7 mins
Batch: 0000/0625 || Epoch: 004/010
Batch: 0100/0625 || Epoch: 004/01

In [54]:
# Testing the language model
# text = "he is"
text = "his family"
input_ids = tokenizer.encode(text, return_tensors='pt')
mask = torch.ones(input_ids.shape)
mask_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1]
model.eval()
with torch.set_grad_enabled(False):

  outputs = model(input_ids=input_ids.to(device), attention_mask=mask.to(device))
  logits = outputs[0]
  logits = logits[:, -1, :]
  filtered_next_token_logits = top_k_top_p_filtering(logits, top_k=5, top_p=1.0)
  probs = F.softmax(filtered_next_token_logits, dim=-1)
  next_token = torch.multinomial(probs, num_samples=1)
  generated = torch.cat([input_ids.to(device), next_token], dim=-1)

  resulting_string = tokenizer.decode(generated.tolist()[0], skip_special_tokens=True)
  print(resulting_string)

his family,
